# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,28.09,83,75,6.17,TO,1675377098
1,1,Rikitea,-23.1203,-134.9692,26.33,70,79,5.96,PF,1675377098
2,2,Burgeo,47.6167,-57.6152,-5.60,76,100,6.58,CA,1675377099
3,3,Kamaishi,39.2667,141.8833,-3.01,62,44,2.33,JP,1675376808
4,4,Leland,34.2563,-78.0447,6.17,94,100,2.06,US,1675376948


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .5    
)

# Display the map plot
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [93]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 5.5) &
    (city_data_df["Humidity"] < 80) &
    (city_data_df["Cloudiness"] < 50),:].dropna(axis=0)

# Display sample data
ideal_city

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Arraial do Cabo,-22.9661,-42.0278,26.98,74,20,4.12,BR,1675377100
34,34,Cabo San Lucas,22.8909,-109.9124,23.12,59,40,2.57,MX,1675377109
49,49,Ixtapa,20.7000,-105.2000,25.13,69,20,3.60,MX,1675377113
109,109,Flinders,-34.5833,150.8552,21.21,51,1,4.92,AU,1675377137
139,139,Tuatapere,-46.1333,167.6833,24.44,61,37,0.24,NZ,1675377149
154,154,Tahoua,14.8888,5.2692,26.99,12,18,2.06,NE,1675377154
162,162,Diego de Almagro,-26.3667,-70.0500,23.94,33,0,3.98,CL,1675377089
169,169,Valparaíso,-33.0393,-71.6273,22.13,60,0,5.14,CL,1675377158
170,170,Tcholliré,8.4022,14.1698,24.09,21,3,1.38,CM,1675377159
179,179,Kaitangata,-46.2817,169.8464,25.28,54,28,1.01,NZ,1675377162


### Step 3: Create a new DataFrame called `hotel_df`.

In [84]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City","Latitude","Longitude","Humidity","Country"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()


/var/folders/b9/y7yy3gjx1vjcj8dmj6zw15kw0000gn/T/ipykernel_5816/1931276726.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Latitude,Longitude,Humidity,Country,Hotel Name
7,Arraial do Cabo,-22.9661,-42.0278,74,BR,
34,Cabo San Lucas,22.8909,-109.9124,59,MX,
49,Ixtapa,20.7000,-105.2000,69,MX,
139,Tuatapere,-46.1333,167.6833,61,NZ,
154,Tahoua,14.8888,5.2692,12,NE,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [94]:
# Set the parameters for the type of search
limit = 20
radius = 1000

# set up a parameters dictionary
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":f"accommodation.hotel"
}


In [95]:

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude from hotel dataframe for row
    latitude =  hotel_df.loc[index, "Latitude"]
    longitude = hotel_df.loc[index, "Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Arraial do Cabo - nearest hotel: No hotel found
Cabo San Lucas - nearest hotel: Comfort Rooms
Ixtapa - nearest hotel: No hotel found
Tuatapere - nearest hotel: No hotel found
Tahoua - nearest hotel: Hotel Bungalow
Diego de Almagro - nearest hotel: No hotel found
Tcholliré - nearest hotel: No hotel found
Kaitangata - nearest hotel: No hotel found
Rong Kwang - nearest hotel: No hotel found
San Cristobal - nearest hotel: Terra Sur
Chon Daen - nearest hotel: No hotel found
San Miguel de Tucumán - nearest hotel: Suites San Luis
São José da Coroa Grande - nearest hotel: Hotel do Sol
Salalah - nearest hotel: Muscat International Hotel
Kissidougou - nearest hotel: Mantisse
Taltal - nearest hotel: Hostería Taltal
Casma - nearest hotel: No hotel found
Gurupi - nearest hotel: No hotel found


,City,Latitude,Longitude,Humidity,Country,Hotel Name
7,Arraial do Cabo,-22.9661,-42.0278,74,BR,No hotel found
34,Cabo San Lucas,22.8909,-109.9124,59,MX,Comfort Rooms
49,Ixtapa,20.7000,-105.2000,69,MX,No hotel found
139,Tuatapere,-46.1333,167.6833,61,NZ,No hotel found
154,Tahoua,14.8888,5.2692,12,NE,Hotel Bungalow
162,Diego de Almagro,-26.3667,-70.0500,33,CL,No hotel found
170,Tcholliré,8.4022,14.1698,21,CM,No hotel found
179,Kaitangata,-46.2817,169.8464,54,NZ,No hotel found
255,Rong Kwang,18.3390,100.3174,44,TH,No hotel found
274,San Cristobal,7.7669,-72.2250,46,VE,Terra Sur


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [99]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .5,
    hover_cols = "all"    
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,index,Country,Hotel Name)